In [2]:
import numpy as np
import pandas as pd
import time
from functools import partial

from scipy.stats import randint, uniform
from sim_lib import simulation

pd.options.mode.chained_assignment = None


In [3]:
def eps_greedy(history: pd.DataFrame, eps: float):
    if uniform.rvs() < eps:
        n = history.shape[0]
        return history.index[randint.rvs(0, n)]

    ctr = history['clicks'] / (history['impressions'] + 10)
    n = np.argmax(ctr)
    return history.index[n]

policy = partial(eps_greedy, eps=0.06)

In [4]:
# seed for homework
seed = 18475
np.random.seed(seed=seed)

start = time.time()
output = simulation(policy, n=200000, seed=seed)
end = time.time()
end - start

1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
50001 impressions have been simulated
60001 impressions have been simulated
70001 impressions have been simulated
80001 impressions have been simulated
90001 impressions have been simulated
100001 impressions have been simulated
110001 impressions have been simulated
120001 impressions have been simulated
130001 impressions have been simulated
140001 impressions have been simulated
150001 impressions have been simulated
160001 impressions have been simulated
170001 impressions have been simulated
180001 impressions have been simulated
190001 impressions have been simulated


356.75614881515503

In [5]:
# baseline regret
output['regret'], output['regret']/output['rounds'],  output['total_banners']

(1540.7609683932544, 0.007703804841966272, 184)

In [6]:
output['history']

,impressions,clicks,lifetime,p
153,18970.0,4199.0,18003.025431,0.220134
162,228.0,26.0,1537.166719,0.113780
172,163.0,32.0,19648.592394,0.219968
173,170.0,18.0,12771.474990,0.122694
180,26.0,3.0,4655.819793,0.020061
182,6.0,0.0,889.624649,0.004621
183,1.0,0.0,15187.163761,0.073886


Реализуем UCB

In [16]:
class UCB:
    def __init__(self, C):                    
        self._C = C
        self._t = 0      
        
    def __call__(self, history: pd.DataFrame):
        self._t += 1
        ctr = history['clicks'] / (history['impressions'] + 1e-6)
        exploration = self._C * (np.log(self._t) / (history['impressions'] + 1e-6)) ** 0.5
        n = np.argmax(ctr + exploration)
        return history.index[n]


In [17]:
for C in [0.001, 0.01, 0.1, 1.0]:
    policy = UCB(C=C)
    # seed for homework
    seed = 18475
    np.random.seed(seed=seed)
    
    output = simulation(policy, n=200000, seed=seed)
    print(f"С = {C}")
    print(f"regret: {output['regret']}, regret/rounds: {output['regret']/output['rounds']}, total banners: {output['total_banners']}")


1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
50001 impressions have been simulated
60001 impressions have been simulated
70001 impressions have been simulated
80001 impressions have been simulated
90001 impressions have been simulated
100001 impressions have been simulated
110001 impressions have been simulated
120001 impressions have been simulated
130001 impressions have been simulated
140001 impressions have been simulated
150001 impressions have been simulated
160001 impressions have been simulated
170001 impressions have been simulated
180001 impressions have been simulated
190001 impressions have been simulated
С = 0.001
regret: 4618.1542180818205, regret/rounds: 0.023090771090409103, total banners: 184
1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been si

Заметим, что лучший результат при C = 0.1, regret становится почти в 10 раз меньше, чем бейзлайн.

Когда мы меняем C мы как раз смотрим на баланс exploration/exploitation, посмотирм еще немного в окрестности 0.1

In [18]:
for C in [0.05, 0.1, 0.2, 0.3]:
    policy = UCB(C=C)
    # seed for homework
    seed = 18475
    np.random.seed(seed=seed)
    
    output = simulation(policy, n=200000, seed=seed)
    print(f"С = {C}")
    print(f"regret: {output['regret']}, regret/rounds: {output['regret']/output['rounds']}, total banners: {output['total_banners']}")


1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
50001 impressions have been simulated
60001 impressions have been simulated
70001 impressions have been simulated
80001 impressions have been simulated
90001 impressions have been simulated
100001 impressions have been simulated
110001 impressions have been simulated
120001 impressions have been simulated
130001 impressions have been simulated
140001 impressions have been simulated
150001 impressions have been simulated
160001 impressions have been simulated
170001 impressions have been simulated
180001 impressions have been simulated
190001 impressions have been simulated
С = 0.05
regret: 249.76800905727052, regret/rounds: 0.0012488400452863526, total banners: 184
1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been si

Оставим в итоге 0.1, он остался лучшим
